In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
import warnings
warnings.filterwarnings("ignore")

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

### Built-in LangChain tools

In [3]:
#!pip install -U wikipedia

In [4]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPL
from langchain_openai import ChatOpenAI

In [5]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [6]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [7]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

/tmp/ipykernel_29510/3609475951.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [8]:
agent("What is 2 to the power of 10?")

/tmp/ipykernel_29510/398535690.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent("What is 2 to the power of 10?")




> Entering new AgentExecutor chain...
I need to calculate 2 to the power of 10.
Action: Calculator
Action Input: 2 ** 10
Observation: Answer: 1024
Thought:I now know the final answer
Final Answer: 1024

> Finished chain.


{'input': 'What is 2 to the power of 10?', 'output': '1024'}

### Wikipedia example

In [9]:
question = "Radiohead is a band famous for which studio albums?"
result = agent(question)



> Entering new AgentExecutor chain...
I should use Wikipedia to find out which studio albums Radiohead is famous for.
Action: wikipedia
Action Input: Radiohead discography
Observation: Page: Radiohead discography
Summary: The English rock band Radiohead have released nine studio albums, one live album, five compilation albums, one remix album, nine video albums, seven EPs, 32 singles, and 48 music videos. Their debut album, Pablo Honey, released in February 1993, reached number 22 in the UK, receiving platinum certifications in the UK and US. Their debut single, "Creep", remains their most successful, entering the top 10 in several countries. Their second album, The Bends, released in March 1995, reached number four in the UK and is certified triple platinum.
Radiohead's third album, OK Computer, was released in May 1997. It remains their most successful album, reaching number one in the UK and Ireland and the top 10 in several other countries. It was certified triple platinum and pr

### Python Agent

In [10]:
agent = create_python_agent(
    llm, 
    tool=PythonREPLTool(),
    verbose=True
)

In [11]:
customer_list = [["Thomas", "Cat"],
                ["John", "Dog"],
                ["Michael", "Fish"],
                ["Sarah", "Bird"],
                ["Jessica", "Lizard"],
                ["Daniel", "Turtle"],
                ["Emily", "Hamster"],
                ["Matthew", "Frog"],
                ]

In [12]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...


/tmp/ipykernel_29510/2161092319.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(f"""Sort these customers by \
Python REPL can execute arbitrary code. Use with caution.


We can use the `sorted()` function in Python to sort the list of customers based on their last name and then first name.
Action: Python_REPL
Action Input: customers = [['Thomas', 'Cat'], ['John', 'Dog'], ['Michael', 'Fish'], ['Sarah', 'Bird'], ['Jessica', 'Lizard'], ['Daniel', 'Turtle'], ['Emily', 'Hamster'], ['Matthew', 'Frog']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
print(sorted_customers)
Observation: [['Sarah', 'Bird'], ['Thomas', 'Cat'], ['John', 'Dog'], ['Michael', 'Fish'], ['Matthew', 'Frog'], ['Emily', 'Hamster'], ['Jessica', 'Lizard'], ['Daniel', 'Turtle']]

Thought:The customers are now sorted by last name and then first name.
Final Answer: [['Sarah', 'Bird'], ['Thomas', 'Cat'], ['John', 'Dog'], ['Michael', 'Fish'], ['Matthew', 'Frog'], ['Emily', 'Hamster'], ['Jessica', 'Lizard'], ['Daniel', 'Turtle']]

> Finished chain.


"[['Sarah', 'Bird'], ['Thomas', 'Cat'], ['John', 'Dog'], ['Michael', 'Fish'], ['Matthew', 'Frog'], ['Emily', 'Hamster'], ['Jessica', 'Lizard'], ['Daniel', 'Turtle']]"

#### View detailed outputs of the chains

In [13]:
from langchain.agents import tool
from datetime import date

In [14]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [15]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
    )

In [16]:
try:
    result = agent("What is the date today?")
except:
    print("Exception on external access")



> Entering new AgentExecutor chain...
I should use the time function to get today's date.
Action: time
Action Input: ""
Observation: 2025-04-20
Thought:I have the current date.
Final Answer: The date today is 2025-04-20.

> Finished chain.


In [17]:
@tool
def combine_name(text: str) -> str:
    """Returns a string with the first and last name of all persons in the customer list"""
    return ", ".join([f"{x[0]} {x[1]}" for x in customer_list])

In [18]:
agent_name= initialize_agent(
    tools + [combine_name],
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [19]:
try:
    result = agent_name("What is the name of the customers in the customer list?")
except:
    print("Exception on external access")



> Entering new AgentExecutor chain...
I should use the combine_name function to get the names of the customers.
Action: combine_name
Action Input: customer list
Observation: Thomas Cat, John Dog, Michael Fish, Sarah Bird, Jessica Lizard, Daniel Turtle, Emily Hamster, Matthew Frog
Thought:I now know the names of the customers in the customer list.
Final Answer: The names of the customers in the customer list are Thomas Cat, John Dog, Michael Fish, Sarah Bird, Jessica Lizard, Daniel Turtle, Emily Hamster, Matthew Frog.

> Finished chain.
